In [ ]:
%pip install gym-trading-env pandas numpy matplotlib stable_baselines3 'shimmy>=0.2.1'

In [ ]:
import gym_trading_env
import gymnasium as gym
import pandas as pd

from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv

In [ ]:
df = pd.read_csv('Data/NIFTY50.csv')
df['DatetimeIndex'] = pd.to_datetime(df['DatetimeIndex'])
df.set_index('DatetimeIndex', inplace=True)
df["feature_pct_change"] = df["close"].pct_change()
df["feature_high"] = df["high"] / df["close"] - 1
df["feature_low"] = df["low"] / df["close"] - 1
df.dropna(inplace= True)

df.head()

In [ ]:
env=gym.make('TradingEnv',df=df,verbose=1,name='NIFTY50',trading_fees = 0.01/100,borrow_interest_rate= 0.0003/100,windows=5)
sum=0
done, truncated = False, False
observation, info = env.reset()
while not done and not truncated:
    # Pick a position by its index in your position list (=[-1, 0, 1])....usually something like : position_index = your_policy(observation)
    position_index = env.action_space.sample() # At every timestep, pick a random position index from your position list (=[-1, 0, 1])
    observation, reward, done, truncated, info = env.step(position_index)
    sum+=reward
    # if reward>0 :
    #     sum+=reward
    # else: 
    #     sum-=reward
print(sum)

# env.save_for_render
# env.unwrapped.save_for_render(dir = "render_logs")

In [ ]:
def env_maker():
    return gym.make('TradingEnv',df=df,verbose=1,name='NIFTY50',trading_fees = 0.01/100,borrow_interest_rate= 0.0003/100)

# Create the environment using make_vec_env
env = DummyVecEnv([env_maker])

# Train the A2C model
model = DQN('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=100000)